# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
# pull the weather csv
# index adjusted with code from https://stackoverflow.com/questions/36519086/how-to-get-rid-of-unnamed-0-column-in-a-pandas-dataframe
weather_df = pd.read_csv("../output_data/cities.csv", index_col=[0])
weather_df.head()

,City,Country,Latitude,Longitude,Temp (°F),Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,bredasdorp,ZA,-34.5322,20.0403,55.40,55.40,82,2,4.61
1,port lincoln,CD,-7.0349,29.7638,72.30,72.30,71,65,8.08
2,mataura,NZ,-46.1927,168.8643,49.30,49.30,78,97,6.71
3,akcakoca,TR,41.0866,31.1162,34.75,34.75,86,53,5.93
4,ponta do sol,PT,38.7333,-28.2667,58.98,58.98,98,100,28.74


In [17]:
weather_df["Max Temp (°F)"].max()

89.01

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=g_key)

In [19]:
# store lat lng into Locations
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# create and use humidity as weight
humidity = weather_df["Humidity (%)"].astype(float)

In [20]:
# create humidity heat map
humidity_fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                dissipating = False, max_intensity = 100,
                                point_radius = 1)

humidity_fig.add_layer(heat_layer)

humidity_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# creating df for ideal weather conditions
weather_ideal_df = weather_df.loc[(weather_df['Max Temp (°F)'] >= 78) & 
                                  (weather_df['Wind Speed (mph)'] <= 15) &
                                  (weather_df['Cloudiness (%)'] <= 33) & 
                                  (weather_df['Humidity (%)'] >= 66) & 
                                  (weather_df['Humidity (%)'] <= 90)].dropna()
#index successfully reset with code from https://stackoverflow.com/questions/27736267/pandas-reset-index-is-not-taking-effect
weather_ideal_df.reset_index(drop=True, inplace=True)
weather_ideal_df

,City,Country,Latitude,Longitude,Temp (°F),Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,arraial do cabo,BR,-22.8362,-46.6460,75.76,78.01,78,0,6.91
1,anloga,GH,5.7947,0.8973,82.40,82.40,83,20,9.22
2,winneba,GH,5.3511,-0.6231,82.40,82.40,83,0,9.22
3,bubaque,GW,11.2833,-15.8333,78.06,78.06,70,23,10.67
4,belmonte,BR,-15.8631,-38.8828,78.80,78.80,73,20,10.36
5,kavaratti,IN,10.5669,72.6420,80.01,80.01,70,12,8.10
6,micheweni,TZ,-4.9667,39.8333,79.74,79.74,82,1,10.54
7,inhambane,MZ,-23.8650,35.3833,78.31,78.31,88,10,4.81
8,kilindoni,TZ,-7.9139,39.6668,81.30,81.30,79,18,13.20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = weather_ideal_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Temp (°F),Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,arraial do cabo,BR,-22.8362,-46.6460,75.76,78.01,78,0,6.91,
1,anloga,GH,5.7947,0.8973,82.40,82.40,83,20,9.22,
2,winneba,GH,5.3511,-0.6231,82.40,82.40,83,0,9.22,
3,bubaque,GW,11.2833,-15.8333,78.06,78.06,70,23,10.67,
4,belmonte,BR,-15.8631,-38.8828,78.80,78.80,73,20,10.36,


In [23]:
# Create hotel variable
hotel = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(hotel_df["City"])):

    lat = hotel_df.loc[city]["Latitude"]
    lng = hotel_df.loc[city]["Longitude"]

    lat_lng = f"{lat},{lng}"

    params = {
        "location": lat_lng, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotel.append(hotel_response["results"][0]["name"])
    except:
        hotel.append("Nearest hotel not found")

In [24]:
# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotel
hotel_df

,City,Country,Latitude,Longitude,Temp (°F),Max Temp (°F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,arraial do cabo,BR,-22.8362,-46.6460,75.76,78.01,78,0,6.91,Pousada da Maricota
1,anloga,GH,5.7947,0.8973,82.40,82.40,83,20,9.22,Pin Drop Hotel
2,winneba,GH,5.3511,-0.6231,82.40,82.40,83,0,9.22,Windy Lodge & Beach Resort
3,bubaque,GW,11.2833,-15.8333,78.06,78.06,70,23,10.67,"Casa Dora, Bubaque"
4,belmonte,BR,-15.8631,-38.8828,78.80,78.80,73,20,10.36,Pousada Monte Carmelo O Pão
5,kavaratti,IN,10.5669,72.6420,80.01,80.01,70,12,8.10,Hotels in Lakshadweep Islands
6,micheweni,TZ,-4.9667,39.8333,79.74,79.74,82,1,10.54,Nearest hotel not found
7,inhambane,MZ,-23.8650,35.3833,78.31,78.31,88,10,4.81,Oceano
8,kilindoni,TZ,-7.9139,39.6668,81.30,81.30,79,18,13.20,Bustani B&B


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [26]:
# Add marker layer ontop of heat map
 # Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = [f"Nearby Hotel: {hotel}" for hotel in hotel])
                                                           
# Add the layer to the map
humidity_fig.add_layer(markers)

# Display figure
humidity_fig


Figure(layout=FigureLayout(height='420px'))